#Scrip 
Realizado para ubicar los datos meustreados en las imágenes

In [19]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

from PIL import Image
from PIL.TiffTags import TAGS
import re
import rasterio
import numpy as np
import math
import matplotlib.pyplot as plt


In [12]:
Sig=pd.read_excel('data/Salida_13_03_23/dataGPS_media.xlsx')
Sig.head()

,Muestra,Tipo,Latitud,Longitud,Presición GPS,Altitud GPS
0,3,4,10.799640,-74.203636,74.0,19.92
1,4,2,10.799836,-74.203743,74.0,15.90
2,5,4,10.799969,-74.203827,75.0,14.12
3,6,6,10.800044,-74.203857,75.0,13.64
4,7,4,10.800036,-74.203859,75.0,15.04


In [8]:
Sig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Muestra        83 non-null     int64  
 1   Tipo           83 non-null     int64  
 2   Latitud        83 non-null     float64
 3   Longitud       83 non-null     float64
 4   Presición GPS  83 non-null     float64
 5   Altitud GPS    83 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 4.0 KB


In [10]:
df = pd.DataFrame(
    {
        "n_muestra":Sig["Muestra"],
        "Tipo": Sig["Tipo"],
        "latitud": Sig["Latitud"],
        "longitud": Sig["Longitud"],
    }
)


# colocando una imagen pra verificar los puntos

In [23]:
imagen='data/Salida_13_03_23/Dron/101FPLAN/DJI_0021.TIF'

img = Image.open(imagen)
# obtenemos los Tags de la metadata y se almacenan en un diccionario
meta_dict = {TAGS[key]: img.tag[key] for key in img.tag_v2}

# Extraemos el indicador xmp
p = meta_dict.get("XMP")
s = p.decode("UTF-8")
# dividimos por el salto de linea y obtenemos una lista
div = s.split("\n")

for ind,recorrido in enumerate(div):
    div[ind]=recorrido.strip() 

usar = div[17]

result = re.search(":(.*)=", div[17])
result.group(1)

result2 = re.search("\"(.*)\"", div[17])
result2.group(1)

print(result.group(1))
print(result2.group(1))
metadiccionario = {}
for ind,recorrido in enumerate(div):
    try:
        metadiccionario[re.search(":(.*)=", div[ind]).group(1)] = re.search("\"(.*)\"", div[ind]).group(1)
    except:
        pass

print(metadiccionario)

centro_x = float(metadiccionario["CalibratedOpticalCenterX"])
centro_y = float(metadiccionario["CalibratedOpticalCenterY"])
altura_vuelo = float(metadiccionario["RelativeAltitude"])
distancia_focal = float(metadiccionario["CalibratedFocalLength"])
angulo_giro = float(metadiccionario["GimbalYawDegree"])
latitud = float(metadiccionario["GpsLatitude"])
longitud = float(metadiccionario["GpsLongitude"])

resolucion = altura_vuelo/distancia_focal
resolucion


# Calcular las coordenadas de los extremos en metros
min_lon = float(metadiccionario["GpsLongitude"]) - (
    float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
max_lon = float(metadiccionario["GpsLongitude"]) + (
    float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
min_lat = float(metadiccionario["GpsLatitude"]) - (
    float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
max_lat = float(metadiccionario["GpsLatitude"]) + (
    float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
print(min_lon, max_lon, min_lat, max_lat)


AbsoluteAltitude
+30.62
{'xmpmeta xmlns:x': 'adobe:ns:meta/', 'RDF xmlns:rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'Description rdf:about': 'DJI Meta Data', 'tiff': 'http://ns.adobe.com/tiff/1.0/', 'exif': 'http://ns.adobe.com/exif/1.0/', 'xmp': 'http://ns.adobe.com/xap/1.0/', 'xmpMM': 'http://ns.adobe.com/xap/1.0/mm/', 'dc': 'http://purl.org/dc/elements/1.1/', 'crs': 'http://ns.adobe.com/camera-raw-settings/1.0/', 'drone-dji': 'http://www.dji.com/drone-dji/1.0/', 'Camera': 'http://pix4d.com/camera/1.0', 'ModifyDate': '2023-03-12', 'CreateDate': '2023-03-12', 'Make': 'DJI', 'Model': 'FC6360', 'format': 'image/TIF', 'AbsoluteAltitude': '+30.62', 'RelativeAltitude': '+24.91', 'GpsLatitude': '10.80031965', 'GpsLongitude': '-74.20439982', 'GimbalRollDegree': '+0.00', 'GimbalYawDegree': '-29.00', 'GimbalPitchDegree': '-89.90', 'FlightRollDegree': '-4.00', 'FlightYawDegree': '-29.10', 'FlightPitchDegree': '+1.90', 'FlightXSpeed': '+0.00', 'FlightYSpeed': '+0.20', 'FlightZSpeed': '